In [ ]:
# Get request 
# Starting url navigates to home page for catalogue 
# Need to navigate to Programs of Study:
# https://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/thecollege/programsofstudy.1.html
# Programs of Study provides a list of links to all departments 
# These links need to be pulled and added to the queue 
# Each department page contains divs that where the course title, number, and description can be pulled 
# (Underneath h3 header)
# Queued URLs need to be popped and then appened to another list for cross reference to avoid visitng the same URL twice
# Crawl initial page
# Soup output

def pull_page_html(starting_url, limiting_domain):
    response = requests.get(starting_url)
    soup = bs(response.txt, "html.parser")
    return(soup)


def go(num_pages_to_crawl, course_map_filename, index_filename):
    '''
    Crawl the college catalog and generates a CSV file with an index.

    Inputs:
        num_pages_to_crawl: the number of pages to process during the crawl
        course_map_filename: the name of a JSON file that contains the mapping
          course codes to course identifiers
        index_filename: the name for the CSV of the index.

    Outputs:
        CSV file of the index index.
    '''

    starting_url = ("http://www.classes.cs.uchicago.edu/archive/2015/winter"
                    "/12200-1/new.collegecatalog.uchicago.edu/index.html")
    limiting_domain = "classes.cs.uchicago.edu"

    pull_page_html(starting_url, limiting_domain)

In [10]:
import queue
import json
import sys
import csv
import re
import bs4
import util
import requests
from bs4 import BeautifulSoup as bs

INDEX_IGNORE = set(['a', 'also', 'an', 'and', 'are', 'as', 'at', 'be',
                    'but', 'by', 'course', 'for', 'from', 'how', 'i',
                    'ii', 'iii', 'in', 'include', 'is', 'not', 'of',
                    'on', 'or', 's', 'sequence', 'so', 'social', 'students',
                    'such', 'that', 'the', 'their', 'this', 'through', 'to',
                    'topics', 'units', 'we', 'were', 'which', 'will', 'with',
                    'yet'])

In [8]:
def pull_page_html(starting_url):
    response = requests.get(starting_url)
    soup = bs(response.text, "html.parser")
    return(soup)

soup = pull_page_html(("http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/index.html"))
soup

<!DOCTYPE html>

<html dir="ltr" lang="en" xml:lang="en">
<head>
<title>The College Catalog 2014-2015 &lt; University of Chicago Catalog</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="2014-2015 College Catalog" name="description"/>
<link href="thecollege/search/opensearch.xml" rel="search" title="University of Chicago Catalog" type="application/opensearchdescription+xml"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<link href="thecollege/favicon.ico" rel="shortcut icon"/>
<link href="thecollege/css/reset.css" rel="stylesheet" type="text/css"/>
<link href="thecollege/css/screen.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="thecollege/css/print.css" media="print" rel="stylesheet" type="text/css"/>
<script src="js/jquery.js" type="text/javascript"></script>
<script src="js/lfjs.js" type="text/javascript"></script>
<script src="js/lfjs_any.js" type="text/javascript"></script>
<

In [ ]:
# link for 'Programs of Study' page is a partial link
# <a href="../programsofstudy.1.html">Programs of Study</a>
# This is a partial URL, and needs to be made into an entire URL 
# Need to use util: convert_if_relative_url
def pull_page_html(domain):
    response = requests.get(domain)
    soup = bs(response.text, "html.parser")
    return(soup)

In [13]:
"""
Utility code
"""
# pylint: disable-msg=invalid-name, broad-except, unused-variable
# pylint: disable-msg=len-as-condition, no-else-return, undefined-variable
# pylint: disable-msg=too-many-return-statements, superfluous-parens
# pylint: disable=R1714
import urllib.parse
import os
import requests
import bs4

######### DO NOT CHANGE THIS CODE  #########

def get_request(url):
    '''
    Open a connection to the specified URL and if successful
    read the data.

    Inputs:
        url: must be an absolute URL

    Outputs:
        request object or None

    Examples:
        get_request("http://www.cs.uchicago.edu")
    '''

    if is_absolute_url(url):
        try:
            r = requests.get(url)
            if r.status_code == 404 or r.status_code == 403:
                r = None
        except Exception:
            # fail on any kind of error
            r = None
    else:
        r = None

    return r


def read_request(request):
    '''
    Return data from request object.  Returns result or "" if the read
    fails..
    '''

    try:
        return request.text.encode('iso-8859-1')
    except Exception:
        print("read failed: " + request.url)
        return ""


def get_request_url(request):
    '''
    Extract true URL from the request
    '''
    return request.url


def is_absolute_url(url):
    '''
    Is url an absolute URL?
    '''
    if url == "":
        return False
    return urllib.parse.urlparse(url).netloc != ""


def remove_fragment(url):
    '''remove the fragment from a url'''

    (url, frag) = urllib.parse.urldefrag(url)
    return url


def convert_if_relative_url(current_url, new_url):
    '''
    Attempt to determine whether new_url is a relative URL and if so,
    use current_url to determine the path and create a new absolute
    URL.  Will add the protocol, if that is all that is missing.

    Inputs:
        current_url: absolute URL
        new_url:

    Outputs:
        new absolute URL or None, if cannot determine that
        new_url is a relative URL.

    Examples:
        convert_if_relative_url("http://cs.uchicago.edu", "pa/pa1.html") yields
            'http://cs.uchicago.edu/pa/pa.html'

        convert_if_relative_url("http://cs.uchicago.edu", "foo.edu/pa.html")
            yields 'http://foo.edu/pa.html'
    '''
    if new_url == "" or not is_absolute_url(current_url):
        return None

    if is_absolute_url(new_url):
        return new_url

    parsed_url = urllib.parse.urlparse(new_url)
    path_parts = parsed_url.path.split("/")

    if len(path_parts) == 0:
        return None

    ext = path_parts[0][-4:]
    if ext in [".edu", ".org", ".com", ".net"]:
        return "http://" + new_url
    elif new_url[:3] == "www":
        return "http://" + new_url
    else:
        return urllib.parse.urljoin(current_url, new_url)


ARCHIVES = ("https://www.classes.cs.uchicago.edu/archive/2015/winter"
            "/12200-1/new.collegecatalog.uchicago.edu/thecollege/archives")
LEN_ARCHIVES = len(ARCHIVES)


ARCHIVES_HTTP = ("http://www.classes.cs.uchicago.edu/archive/2015/winter"
            "/12200-1/new.collegecatalog.uchicago.edu/thecollege/archives")
LEN_ARCHIVES_HTTP = len(ARCHIVES_HTTP)


def is_url_ok_to_follow(url, limiting_domain):
    '''
    Inputs:
        url: absolute URL
        limiting domain: domain name

    Outputs:
        Returns True if the protocol for the URL is HTTP, the domain
        is in the limiting domain, and the path is either a directory
        or a file that has no extension or ends in .html. URLs
        that include an "@" are not OK to follow.

    Examples:
        is_url_ok_to_follow("http://cs.uchicago.edu/pa/pa1", "cs.uchicago.edu")
            yields True

        is_url_ok_to_follow("http://cs.cornell.edu/pa/pa1", "cs.uchicago.edu")
            yields False
    '''

    if "mailto:" in url:
        return False

    if "@" in url:
        return False

    if url[:LEN_ARCHIVES] == ARCHIVES or url[:LEN_ARCHIVES_HTTP] == ARCHIVES_HTTP:
        return False

    parsed_url = urllib.parse.urlparse(url)
    if parsed_url.scheme != "http" and parsed_url.scheme != "https":
        return False

    if parsed_url.netloc == "":
        return False

    if parsed_url.fragment != "":
        return False

    if parsed_url.query != "":
        return False

    loc = parsed_url.netloc
    ld = len(limiting_domain)
    trunc_loc = loc[-(ld+1):]
    if not (limiting_domain == loc or (trunc_loc == "." + limiting_domain)):
        return False

    # does it have the right extension
    (filename, ext) = os.path.splitext(parsed_url.path)
    return (ext == "" or ext == ".html")


def is_subsequence(tag):
    '''
    Does the tag represent a subsequence?
    '''
    return isinstance(tag, bs4.element.Tag) and 'class' in tag.attrs \
        and tag['class'] == ['courseblock', 'subsequence']


def is_whitespace(tag):
    '''
    Does the tag represent whitespace?
    '''
    return isinstance(tag, bs4.element.NavigableString) and (tag.strip() == "")


def find_sequence(tag):
    '''
    If tag is the header for a sequence, then
    find the tags for the courses in the sequence.
    '''
    rv = []
    sib_tag = tag.next_sibling
    while is_subsequence(sib_tag) or is_whitespace(tag):
        if not is_whitespace(tag):
            rv.append(sib_tag)
        sib_tag = sib_tag.next_sibling
    return rv


In [ ]:

def navigate_to_programs_of_study(starting_url):
    response = read_request(get_request(starting_url))
    soup = bs(response, "html.parser")
    programs_of_study = soup.find('a', attrs = {'href', })
    return(soup)


programs_of_study_page = navigate_to_programs_of_study("http://www.classes.cs.uchicago.edu/archive/2015/winter/12200-1/new.collegecatalog.uchicago.edu/index.html")
programs_of_study_page


<!DOCTYPE html>

<html dir="ltr" lang="en" xml:lang="en">
<head>
<title>The College Catalog 2014-2015 &lt; University of Chicago Catalog</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="2014-2015 College Catalog" name="description"/>
<link href="thecollege/search/opensearch.xml" rel="search" title="University of Chicago Catalog" type="application/opensearchdescription+xml"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<link href="thecollege/favicon.ico" rel="shortcut icon"/>
<link href="thecollege/css/reset.css" rel="stylesheet" type="text/css"/>
<link href="thecollege/css/screen.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="thecollege/css/print.css" media="print" rel="stylesheet" type="text/css"/>
<script src="js/jquery.js" type="text/javascript"></script>
<script src="js/lfjs.js" type="text/javascript"></script>
<script src="js/lfjs_any.js" type="text/javascript"></script>
<

In [ ]:
def navigate_to_programs_of